# Example: NL Assortment Optimization

This notebook demonstrates how to use the algorithm implemented in this repository for assortment optimization under the **Nested Logit (NL)** choice model.

It shows how to:
1. Configure and run assortment optimization for an unconstrained scenario.
2. Run an efficient optimization heuristic (RO) specifically designed for the NL model.
3. Interpret the results provided by the algorithms.

## 1. Import Required Modules


In [ ]:
import numpy as np  
import sys
import os


notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)  # get the project root directory
sys.path.append(project_root)

from generator.nl_data_generator import *
from generator.constraint import *
from models.nl_functions import *
from heuristic.nl_heuristic import *

print("✅ Modules imported successfully.")

✅ Modules imported successfully.


## 2. Unconstrained Assortment Optimization

In this section, we solve the assortment optimization problem without any additional constraints. 

### 2.1: Define Experiment Configuration and Generate Data
- There are many data generation methods in generator.nl_data_generator, you can choose and give the corresponding parameters
- The random seed is fixed to ensure that the results are reproducible.
- We found that performance was poor for certain random seeds, so we chose these seeds to demonstrate the performance of the heuristic algorithm.

In [ ]:
# --- Configuration ---
m = 5 # the number of nests
n = 25  # the number of products for each nest
gamma_range = [2,3] #the range for the dissimilarity parameter
full_capture = False # whether the no-purchase option is fully captured

worst_seeds = [52, 95, 84, 45, 87] # define the worst-performing seeds



# --- Data Generation --
# Generate the data for the NL model
# The seed is fixed to ensure the generated data is always the same.
data = nl_data_vi0_uniform01(m, n, gamma_range, full_capture, worst_seeds[0])
    

# The revenue function is what we want to maximize.
# There is to create two versions: one for the GPU (if available) and one for the CPU.
revenue_fn_cpu = get_revenue_function_nl(data)


We can inspect the generated data to understand the components of the NL model:
- v: A matrix representing the utility of each product in each nest, shape (m,n).
- price: The price of each product, shape (m,n).
- gamma: The dissimilarity parameter for each nest.
- v0: The utility of the no-purchase option.
- vi0: The utility of the no-purchase option within nest.

In [7]:
np.set_printoptions(suppress=True)
print("the utility :",np.round(data.v,2))
print("the price :",np.round(data.price,2))
print("the no-purchase option utility :",data.v0)
print("within-nest no-purchase utility :",np.round(data.vi0,2))
print("the dissimilarity parameter :",np.round(data.gamma,2))

the utility : [[ 0.19  0.36  0.45  0.89  1.17  1.8   2.18  2.25  3.56  4.29  5.18  5.35
   5.57  5.96  6.82  7.06  7.32  7.67  8.46  8.92  9.23  9.85 10.66 11.22
  11.35]
 [ 0.03  0.19  0.37  2.92  3.28  3.33  3.39  4.7   4.79  4.79  5.04  5.42
   5.64  5.92  6.3   6.54  6.56  6.69  6.77  7.54  7.76  7.96  8.42  8.94
  10.3 ]
 [ 0.68  0.88  1.38  1.5   1.78  2.56  3.38  3.41  3.56  3.64  4.4   4.57
   5.15  5.18  5.42  5.64  6.26  6.29  6.59  7.94  9.24  9.49  9.64  9.87
   9.98]
 [ 0.54  0.72  0.85  0.86  2.94  3.18  3.34  4.4   4.67  4.71  5.88  6.
   6.1   6.7   7.07  7.45  8.25  8.25  8.29  8.77  8.96  9.67  9.9  10.05
  10.13]
 [ 0.34  0.38  1.94  1.94  2.04  2.1   2.66  3.05  3.45  3.51  3.96  4.38
   6.1   6.54  6.6   6.8   6.86  7.25  7.51  8.82  9.    9.84 10.19 10.74
  10.95]]
the price : [[ 8.16 10.88  6.99  9.3   5.77  7.38  5.74  5.43  3.18  3.93  3.08  2.92
   1.69  0.51  1.33  1.72  1.85  0.85  0.4   0.06  0.63  0.13  0.    0.05
   0.05]
 [11.77  8.01  7.1   5.2   3.89  

### 2.2: Perform Optimization
We wWe will now use a specialized revenue-ordered heuristic to find the optimal assortment.

- Revenue-Ordered Heuristic: A algorithm based on linear programming, where each nest $i$ contains the $k_i$ highest-revenue products<sup>[1]</sup>. It is specifically designed for the unconstrained problem under the NL model. It does not support additional constraints.



In [8]:
# This function implements the revenue-ordered heuristic for the NL model.
# It returns the optimal assortment as a binary matrix.
assortment = revenue_order_nl(m,n,data.v,data.price,data.v0,data.vi0,data.gamma)
# Calculate the revenue of the assortment found by the heuristic.
rev_order = revenue_fn_cpu(assortment)[0]
print(f"The revenue obtained by RO: {rev_order}")
print(f"The optimal assortment obtained by RO: {assortment.reshape(m,n)}")

The revenue obtained by RO: 6.3131100815370305
The optimal assortment obtained by RO: [[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


# References

[1] Davis J M, Gallego G, Topaloglu H. Assortment optimization under variants of the nested logit model[J]. Operations Research, 2014, 62(2): 250-273.